# Spårbredden hos en CD-skiva

Medlaboranter: Pontus Liedgren  
Handledare: Sanna Bodemyr  
Tullinge gymnasium  
2021-03-22

## Syfte

Syftet med denna undersökning är att bestämma spårbredden på en CD-skiva.

## Frågeställningar

Vilken spårbredd har CD-skivan?

## Reflektion/diskussion

För att bestämma spårbredden hos en CD-skiva går det att använda en laserpekare vars våglängd är känd (att ta reda på våglängden hos en laserpekare med okänd våglängd går att göra i en separat laboration), CD-skiva, linjal och stativmaterial.

In [1]:
# importera bibliotek
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# läs in laborationsdata
df = pd.read_csv("data.csv")
df # visa data som antecknades under labben

,Wavelength laser (nm),n,x (m),L (m)
0,663,1,0.282,0.287
1,663,1,0.625,0.648
2,631,1,0.604,0.648
3,631,1,0.302,0.323
4,631,2,1.030,0.323


In [3]:
# funktion som beräknar spårbredden utifrån mätvärden i en given rad
def calculate(row):
    angle = np.arctan((row["x (m)"]/2)/row["L (m)"])
    
    # enligt d * sin(alpha) = n * lambda
    # så kan vi beräkna d, gitterkonstanten eller spårbredden, på detta sätt
    d = (row["n"] * row["Wavelength laser (nm)"]) / np.sin(angle)

    return d

# gitterkonstanten
df[f"Track width (nm)"] = df.apply(lambda row: calculate(row), axis=1)

# skriv ut tabellen igen så vi kan se den nya kolumnen
df

,Wavelength laser (nm),n,x (m),L (m),Track width (nm)
0,663,1,0.282,0.287,1503.578386
1,663,1,0.625,0.648,1526.314267
2,631,1,0.604,0.648,1493.752880
3,631,1,0.302,0.323,1489.966265
4,631,2,1.030,0.323,1489.673457


## Diskussion

$F_g = m \cdot g$